In [19]:
from faker import Faker
import pandas as pd
import random

# Instantiate Faker
fake = Faker()

# Define possible HTTP methods and response codes
http_methods = ['GET', 'POST', 'PUT', 'DELETE', 'PATCH']
response_codes = [200, 201, 202, 204, 400, 401, 403, 404, 500]

# Prepare empty lists to hold data
source_ips = []
user_agents = []
urls = []
methods = []
responses = []

# Generate unique IP addresses
num_unique_ips = 20  # Adjust as needed
unique_ips = [fake.ipv4() for _ in range(num_unique_ips)]

# Generate a list of unique URLs for each IP
ip_to_urls = {ip: [fake.url() for _ in range(3)] for ip in unique_ips}

# Generate data
num_records_per_ip = 7  # Adjust as needed
for ip in unique_ips:
    for _ in range(random.randint(3, 10)):  # Each IP is repeated 6-20 times
        source_ips.append(ip)
        user_agents.append(fake.user_agent())
        # Choose a URL from the pre-generated list for this IP
        urls.append(random.choice(ip_to_urls[ip]))
        methods.append(random.choice(http_methods))
        responses.append(random.choice(response_codes))

# Create a DataFrame
df = pd.DataFrame({
    'source_ip': source_ips,
    'user_agent': user_agents,
    'url': urls,
    'http_method': methods,
    'response_code': responses
})

# Save to CSV
df.to_csv('http_requests.csv', index=False)

In [20]:
import networkx as nx
import pandas as pd

df

,source_ip,user_agent,url,http_method,response_code
0,94.82.28.251,Mozilla/5.0 (iPod; U; CPU iPhone OS 3_1 like M...,https://www.walker.info/,PATCH,204
1,94.82.28.251,Mozilla/5.0 (Linux; Android 2.3) AppleWebKit/5...,http://www.stephens-simpson.info/,GET,401
2,94.82.28.251,Mozilla/5.0 (Macintosh; PPC Mac OS X 10_8_9) A...,http://www.stephens-simpson.info/,PUT,202
3,94.82.28.251,Opera/8.41.(X11; Linux i686; ht-HT) Presto/2.9...,http://www.stephens-simpson.info/,PUT,401
4,94.82.28.251,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,http://www.stephens-simpson.info/,GET,200
...,...,...,...,...,...
106,141.233.239.160,Mozilla/5.0 (Linux; Android 8.0.0) AppleWebKit...,http://montoya-hebert.com/,POST,201
107,141.233.239.160,Mozilla/5.0 (compatible; MSIE 8.0; Windows 98;...,https://www.allen.net/,PATCH,204
108,141.233.239.160,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,https://www.allen.net/,PATCH,400
109,141.233.239.160,Mozilla/5.0 (compatible; MSIE 8.0; Windows 98;...,http://montoya-hebert.com/,PUT,403


In [24]:

# Create an empty Directed Graph
G = nx.DiGraph()

# Add nodes and edges with their attributes
for index, row in df.iterrows():
    # Add nodes
    G.add_node(row['source_ip'], node_type='ip')
    G.add_node(row['user_agent'], node_type='user_agent')
    G.add_node(row['url'], node_type='url')

    # Add edges (relationships)
    G.add_edge(row['source_ip'], row['url'], method=row['http_method'], response_code=row['response_code'])
    G.add_edge(row['url'], row['user_agent'])
    G.add_edge(row['source_ip'], row['user_agent'])

# Print the graph
print(nx.info(G))

DiGraph with 182 nodes and 274 edges


C:\Users\i00646629\AppData\Local\Temp\ipykernel_23452\575551315.py:17: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(G))


In [29]:
import plotly.graph_objects as go
import networkx as nx

# Assuming you have a NetworkX graph G

pos = nx.spring_layout(G)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2),
    text=list(G.nodes()), 
    textposition="top center")

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Attackers connection graph',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()